In [2]:
import pandas as pd
import numpy as np
import tensorflow
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LearningRateScheduler
import pickle

In [3]:
df = pd.read_csv("../mortgage_encoded_final.csv")

In [4]:
df.head()

,county_code,census_tract,action_taken,preapproval,loan_type,loan_purpose,lien_status,open-end_line_of_credit,loan_amount,loan_term,...,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_one_to_four_family_homes,tract_median_age_of_housing_units,encoded_loan_limit,encoded_debt_ratio,encoded_state_code,encoded_dwelling_category,encoded_applicant_age,encoded_co-applicant_age
0,34001.0,3.400101e+10,2,2,1,31,1,1,365000.0,360.0,...,71600,123,1038,43,0,5,32,1,3,3
1,42017.0,4.201710e+10,2,2,1,31,2,1,155000.0,360.0,...,107900,138,2012,18,0,3,39,1,2,2
2,42041.0,4.204101e+10,2,2,1,31,1,1,195000.0,360.0,...,75200,93,1686,26,0,3,39,1,4,4
3,34021.0,3.402100e+10,2,2,1,31,1,1,155000.0,360.0,...,98900,104,1850,50,0,4,32,1,4,3
4,42133.0,4.213302e+10,2,2,1,31,2,1,105000.0,360.0,...,74300,96,1590,37,0,2,39,1,3,3


In [5]:
X= df.drop("action_taken", axis=1)
y=df["action_taken"]
print(X.shape, y.shape)

(898438, 45) (898438,)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=55)
#X_train, y_train = X[:90000], y_train[:90000]

In [8]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
#Step 1: Label-encode data set
le = LabelEncoder()
le.fit(y_train)
encoded_y_train = le.transform(y_train)
encoded_y_test = le.transform(y_test)

In [10]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

## Deep Learning Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dropout

In [12]:
# Create model and add layers
#Dense(10, activation='relu', kernel_initializer='lecun_normal')

model = Sequential()
model.add(Dense(units=164, activation='tanh', input_dim=45))
model.add(Dense(units=128, activation='tanh'))
model.add(Dropout(.5))
model.add(Dense(units=32, activation='tanh'))
model.add(Dense(units=8, activation='tanh'))
model.add(Dense(units=2, activation='softmax'))

In [13]:
# Compile and fit the model
sgd = tensorflow.keras.optimizers.SGD(learning_rate=.03)

model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 164)               7544      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               21120     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 18        
Total params: 33,074
Trainable params: 33,074
Non-trainable params: 0
____________________________________________________

In [ ]:
initial_learning_rate = .03
epochs =300
decay = initial_learning_rate/epochs

def lr_time_based_decay(epoch, lr):
    return lr * 1/(1 + decay * epoch)

filepath = "mortgage_13_1_7_final_final"

model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_best_only=True,
    monitor='val_accuracy',
    save_weights_only = True,
    verbose=1
    )

callback = tensorflow.keras.callbacks.EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=300,
    validation_split=.4,
    shuffle=True,
    callbacks=[LearningRateScheduler(lr_time_based_decay, verbose=1), callback, model_checkpoint_callback]
)

Epoch 1/300

Epoch 00001: LearningRateScheduler setting learning rate to 0.029999999329447746.
12635/12635 [==============================] - 51s 4ms/step - loss: 0.4459 - accuracy: 0.8126 - val_loss: 0.4377 - val_accuracy: 0.8115

Epoch 00001: val_accuracy improved from -inf to 0.81154, saving model to mortgage_13_1_7_final_final
Epoch 2/300

Epoch 00002: LearningRateScheduler setting learning rate to 0.029996999629484798.
12635/12635 [==============================] - 52s 4ms/step - loss: 0.4349 - accuracy: 0.8143 - val_loss: 0.4397 - val_accuracy: 0.8113

Epoch 00002: val_accuracy did not improve from 0.81154
Epoch 3/300

Epoch 00003: LearningRateScheduler setting learning rate to 0.029991002270303425.
12635/12635 [==============================] - 33s 3ms/step - loss: 0.4317 - accuracy: 0.8145 - val_loss: 0.4318 - val_accuracy: 0.8115

Epoch 00003: val_accuracy did not improve from 0.81154
Epoch 4/300

Epoch 00004: LearningRateScheduler setting learning rate to 0.02998200815093168.

12635/12635 [==============================] - 60s 5ms/step - loss: 0.4093 - accuracy: 0.8160 - val_loss: 0.4088 - val_accuracy: 0.8144

Epoch 00027: val_accuracy did not improve from 0.81441
Epoch 28/300

Epoch 00028: LearningRateScheduler setting learning rate to 0.02888816802537583.
12635/12635 [==============================] - 59s 5ms/step - loss: 0.4090 - accuracy: 0.8163 - val_loss: 0.4084 - val_accuracy: 0.8139

Epoch 00028: val_accuracy did not improve from 0.81441
Epoch 29/300

Epoch 00029: LearningRateScheduler setting learning rate to 0.02880750679439599.
12635/12635 [==============================] - 56s 4ms/step - loss: 0.4087 - accuracy: 0.8160 - val_loss: 0.4112 - val_accuracy: 0.8133

Epoch 00029: val_accuracy did not improve from 0.81441
Epoch 30/300

Epoch 00030: LearningRateScheduler setting learning rate to 0.02872420576850419.
12635/12635 [==============================] - 96s 8ms/step - loss: 0.4086 - accuracy: 0.8161 - val_loss: 0.4094 - val_accuracy: 0.8138

Ep

12635/12635 [==============================] - 97s 8ms/step - loss: 0.4052 - accuracy: 0.8173 - val_loss: 0.4076 - val_accuracy: 0.8139

Epoch 00055: val_accuracy did not improve from 0.81525
Epoch 56/300

Epoch 00056: LearningRateScheduler setting learning rate to 0.02572546731115632.
12635/12635 [==============================] - 77s 6ms/step - loss: 0.4050 - accuracy: 0.8177 - val_loss: 0.4056 - val_accuracy: 0.8149

Epoch 00056: val_accuracy did not improve from 0.81525
Epoch 57/300

Epoch 00057: LearningRateScheduler setting learning rate to 0.025582206772615156.
12635/12635 [==============================] - 74s 6ms/step - loss: 0.4049 - accuracy: 0.8174 - val_loss: 0.4062 - val_accuracy: 0.8151

Epoch 00057: val_accuracy did not improve from 0.81525
Epoch 58/300

Epoch 00058: LearningRateScheduler setting learning rate to 0.025437215239956398.
12635/12635 [==============================] - 78s 6ms/step - loss: 0.4048 - accuracy: 0.8177 - val_loss: 0.4068 - val_accuracy: 0.8148



In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history['val_accuracy'])
plt.show()

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history['val_loss'])
plt.show()

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
model.save("mortgage_test_13_1_4_final.h5", save_format="h5")

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))